<a href="https://colab.research.google.com/github/jeevannani1705/Project-with-ML/blob/main/Random_Forest_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Code to read csv file into colaboratory:
!pip install sentence-transformers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
import sentence_transformers
from sentence_transformers import SentenceTransformer, models
data_set=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROJECT WITH ML 2022-JEEVAN/global-headline-only.csv')[['event_type', 'cleaned']]
data_set.head(1)

,event_type,cleaned
0,Explosions/Remote violence,regime forces shelled Arnaba town in Idleb cou...


In [5]:
print (f"Shape of original data set is {data_set.shape}")

Shape of original data set is (1034527, 2)


In [6]:
df=data_set.sample(100000).reset_index().drop(['index'], axis = 1)
df.head(1)

,event_type,cleaned
0,Battles,ProHouthi forces claimed to have killed at lea...


In [7]:
#del data_set

In [8]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [9]:
train = model.encode(df.cleaned, device='cuda')

In [10]:
from sklearn import preprocessing
# features_trainl encode the target variable
encoder = preprocessing.LabelEncoder()
df["event_type_num"] = encoder.fit_transform(df["event_type"])

In [11]:
df_features = pd.DataFrame()
df_features['cleaned_emb'] = [i for i in train] 
df_features['event_type'] = df["event_type_num"]
df_features.head(3)

,cleaned_emb,event_type
0,"[-0.0018029148, 0.26510158, 0.009531988, 0.097...",0
1,"[-0.1253956, -0.22168873, -0.150455, 0.4932399...",0
2,"[-0.111128405, 0.2981903, -0.014155166, -0.117...",1


In [12]:
 df_features.to_csv("./features_set.csv")

In [13]:
from sklearn.decomposition import PCA
import time
pca_model = PCA(n_components=50)
start = time.process_time()
pca_model.fit(train)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))
print(time.process_time() - start)

Sum of variance ratios:  0.5966195062381492
16.53647529500006


In [14]:
x_comps = pca_model.transform(train)
x_comps.shape

(100000, 50)

**Model Training**

Random Forest

In [15]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_comps,df_features['event_type'],test_size=0.2,random_state=42)

In [16]:
# More algorithms!!!!
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)
print("Score of RFC",rfc.score(x_test,y_test))



Score of RFC 0.892


In [20]:
rfc.score(x_test,y_test)

0.892

In [23]:
y_pred = rfc.predict(x_test)

In [24]:
print(y_pred)
print(y_test)

[0 0 2 ... 0 4 2]
75721    0
80184    0
19864    2
76699    5
92991    2
        ..
32595    0
29313    0
37862    0
53421    4
42410    2
Name: event_type, Length: 20000, dtype: int64


In [25]:
from sklearn.metrics import classification_report

print(classification_report(y_true= y_test, y_pred= y_pred))

              precision    recall  f1-score   support

           0       0.88      0.89      0.89      4786
           1       0.92      0.88      0.90      2547
           2       0.95      0.98      0.96      7084
           3       0.83      0.65      0.73      1606
           4       0.85      0.72      0.78      1082
           5       0.80      0.87      0.83      2895

    accuracy                           0.89     20000
   macro avg       0.87      0.83      0.85     20000
weighted avg       0.89      0.89      0.89     20000



K FOLD

In [26]:
# Making the Confusion Matrix 
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[4276   82    0   39   24  365]
 [ 245 2254    0    0   22   26]
 [   5    2 6964   71   26   16]
 [  54   11  321 1042   29  149]
 [  63   66   55   26  780   92]
 [ 203   24   23   81   40 2524]]


In [28]:
# Applying k-Fold Cross Validation 
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = rfc, X = x_train, y = y_train, cv = 10) 
accuracies.mean() 
accuracies.std()

0.004433818895715075